In [2]:
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from youtube_transcript_api import YouTubeTranscriptApi,TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnableLambda,RunnableParallel,RunnablePassthrough



In [3]:
video_id="iv-5mZ_9CPY"
ytt_api=YouTubeTranscriptApi()
try:
    transcript_list=ytt_api.fetch(video_id,languages=["en","hi"])
    transcript=" ".join(chunk.text for chunk in transcript_list)
except TranscriptsDisabled:
    print("Not found")
    

In [7]:
len(transcript)

37476

In [8]:
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,
                               chunk_overlap=400)

In [9]:
chunk=splitter.create_documents([transcript])

In [189]:
len(chunk)

62

In [10]:
embedding=OpenAIEmbeddings()
vector_store=Chroma.from_documents(chunk,
    embedding=embedding,
    )

In [11]:
retriever=vector_store.as_retriever(search_type="mmr",
                                    search_kwargs={"k":3})

In [12]:
prompt=PromptTemplate(
    template="""If the context is present in the given text then only give discuss about the question. 
    If the context is not present then just Say I dont know 
    {context}
    
    {question}
    """,
    input_variables=["context","question"]
)

In [13]:
llm=ChatOpenAI()


In [14]:
def formated_doc(docs):
    context_text="\n\n".join(doc.page_content for doc in docs)
    return context_text


In [18]:
parallel_chain=RunnableParallel(
    {
        "context": retriever | RunnableLambda(formated_doc), 
        "question":RunnablePassthrough()
    }
)

In [15]:
prompt1=PromptTemplate(
    template= "If the spelling in the sentence is mistake then correct it\n {sentence} \n",
    input_variables=["sentence"]
)

In [16]:
prompt2=PromptTemplate(
    template= "If the grammar is mistake in the sentence is mistake then correct it\n {sentence} \n",
    input_variables=["sentence"]
)


'Summarize the video.'

In [19]:
parser=StrOutputParser()
chain=prompt1| llm |parser| prompt2|llm| parser|parallel_chain |prompt|llm| parser
chain.invoke("vido summariz")

'The text discusses the use of diffusion models and negative prompts in video generation using language as input. This new approach allows for steering the diffusion process and creating lifelike images and videos without the need for traditional art skills or animation software. The author also mentions their decision to have guest creators make videos for their channel while they are on paternity leave.'